In [1]:
from dataprocess.kdd_cup99 import KDD_CUP_99_DataLoader
from net.CNN_KDD import CNN_KDD

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [3]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
dataset = KDD_CUP_99_DataLoader('E:/DataSets/kddcup.data', 256)
net = CNN_KDD(41,23,3).to(device)
epoch = 100

In [5]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=1e-2)

In [6]:
writer1 = SummaryWriter('log/exp')

In [16]:
for t in range(epoch):
    running_loss = 0
    for step, (x, y) in enumerate(dataset):
        x = x.to(device)

        y = F.one_hot(y.long(), num_classes=23).float()
        y = y.to(device)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度

        y_pred = net(x)
        loss = criterion(y_pred, y)
        loss.backward() # loss反向传播
        optimizer.step() # 反向传播后参数更新 
        
        # visualize loss
        running_loss += loss.item()
    # ...log the running loss
    writer1.add_scalar('training loss', running_loss, t)
    if t % 10 == 9:
        checkpoint = {"net": net.state_dict(), 'optimizer':optimizer.state_dict(), "epoch": t}
        if not os.path.isdir("/Transfer-learning-IDS/history/CNN_KDD/checkpoint"):
            os.makedirs("/Transfer-learning-IDS/history/CNN_KDD/checkpoint")
        torch.save(checkpoint, '/Transfer-learning-IDS/history/CNN_KDD/checkpoint/ckpt_best_%s.pth' %(str(t)))
writer1.close()